In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime
import matplotlib.dates as mdates
import os, sys
import pandas as pd
from sklearn.metrics import r2_score
from tqdm import tqdm
import pandas as pd


def MakeDataFromText(filepath:str):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234

r1 = MakeDataFromText("Temperature 50p5/RSSI_66212.csv")
t1 = MakeDataFromText("Temperature 50p5/tags_66212.csv")
r2 = MakeDataFromText("Temperature 50p8/RSSI_95451.csv")
t2 = MakeDataFromText("Temperature 50p8/tags_95451.csv")
r3 = MakeDataFromText("40p7/RSSI_56307.csv")
t3 = MakeDataFromText("40p7/tags_56307.csv")
r4 = MakeDataFromText("56p2 Data 1/RSSI_44986.csv")
t4 = MakeDataFromText("56p2 Data 1/tags_44986.csv")
r5 = MakeDataFromText("56p2 Data 2/RSSI_44926.csv")
t5 = MakeDataFromText("56p2 Data 2/tags_44926.csv")
t6 = MakeDataFromText("55p5/tags_69303.csv")
r6 = MakeDataFromText("55p5/RSSI_69303.csv")
t7 = MakeDataFromText("23p7/tags_13318.csv")
r7 = MakeDataFromText("23p7/RSSI_13318.csv")
t8 = MakeDataFromText("23p2/tags_60650.csv")
r8 = MakeDataFromText("23p2/RSSI_60650.csv")
t9 = MakeDataFromText('23p3_1/tags_13318.csv')
r9 = MakeDataFromText('23p3_1/RSSI_13318.csv')

temp23p2 = MakeDict([t8])
rssi23p2 = MakeDict([r8])
temp50p5 = MakeDict([t1])
rssi50p5 = MakeDict([r1])
temp50p8 = MakeDict([t2])
rssi50p8 = MakeDict([r2])
temp40p7 = MakeDict([t3])
rssi40p7 = MakeDict([r3])
temp56p2_1 = MakeDict([t4])
rssi56p2_1 = MakeDict([r4])
temp56p2_2 = MakeDict([t5])
rssi56p2_2 = MakeDict([r5])
temp55p5 = MakeDict([t6])
rssi55p5 = MakeDict([r6])
temp23p7 = MakeDict([t7])
rssi23p7 = MakeDict([r7])
temp23p3 = MakeDict([t9])
rssi23p3 = MakeDict([r9])

In [ ]:
LABEL = ["23.2", "23.7", "40.7", "50.5", "50.8", "55.5", "56.2 #1", "56.2 #2"]
TEMP = [temp23p2, temp23p7, temp40p7, temp50p5, temp50p8, temp55p5, temp56p2_1, temp56p2_2]
RSSI = [rssi23p2, rssi23p7, rssi40p7, rssi50p5, rssi50p8, rssi55p5, rssi56p2_1, rssi56p2_2]
FITCOLOR = ["#ff0000", "#ff6f00", "#ff14d0", "#fa0505", "#f07e05", "#fa0505"]
RAWCOLOR = ["#0DA995", "#0024c2", "#0F78BC", "#008ec2", "#110063", "#008ec2"]
TAG = "2B42"

In [ ]:
def Actual(mt, mr, tag):
    """
    """
    coef = {
        '2B42': 0.032,
        '3943': 0.037,
        '3242': 0.048,
    }

    at = []
    for r,t in zip(mr, mt):
        at.append( coef[tag] * (10.0-r) + t )
    
    return np.array(at)


def TimeMoveAvg(time, data, window):
    """
    """
    every, avgtime, avgdata = [], [], []
    t0 = time[0]
    while t0 < time[-1]:
        every.append([t0, t0 + datetime.timedelta(seconds=window)])
        t0 += datetime.timedelta(seconds=1)
    
    for j in range(len(every)): 
        d_ = []
        for ti,da in zip(time, data):
            if ti>=every[j][0] and ti<every[j][1]:
                d_.append(da)
        
        if len(d_)>0:
            avgdata.append(sum(d_)/len(d_))
            avgtime.append(every[j][0])
    
    return np.array(avgtime), np.array(avgdata)

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6, 1, figsize=(13, 25))
axs = [ax1, ax2, ax3, ax4, ax5, ax6]

for u in range(6):
    for k,v in TEMP[u].items():
        if k[-4:]==TAG:
            st, sr = sorted( TEMP[u][k].items() ), sorted( RSSI[u][k].items() )
            dt, tt, dr = [ j[1] for j in st ], [ j[0] for j in st ], [ j[1] for j in sr ]

            newt = []
            newr = []
            newtime = []
            for r_,t_,x__ in zip(dr, dt, tt):
                if r_<=22:
                    newt.append(t_)
                    newr.append(r_)
                    newtime.append(x__)

            newtime, newr, newt = zip( *sorted( zip(newtime, newr, newt) ) )

            axs[u].plot(newtime, newt, color='#0d8aff', label='Raw Measured Temp', linewidth=0.75)
            # axs[u].plot(newtime, Actual(newt, newr, TAG), color='#EB0203', label='Actual Temp', linewidth=0.9)

            

            measured_x, measured_y = TimeMoveAvg(newtime, newt, 10)
            actual_x, actual_y     = TimeMoveAvg(newtime, Actual(newt, newr, TAG), 10)

            print(
                LABEL[u], ": ",
                "Raw ", round(np.mean(newt),3),
                "Act ", round(np.mean(Actual(newt, newr, TAG)),3),
                "Mov ", round(np.mean(actual_y), 3)
            )

            print(
                LABEL[u], ": ",
                "Raw std", round(np.std(newt),3),
                "Act std", round(np.std(Actual(newt, newr, TAG)),3),
                "Mov std", round(np.std(actual_y), 3)
            )
                
            # axs[u].plot(measured_x, measured_y, color='#021BE8', label='Measured Temp', linewidth=0.9)
            axs[u].plot(actual_x, actual_y,     color='#EB1213', label='Actual Temp, 10s Avg, 9s overlap with RSSI<21',   linewidth=1.0)

            axs[u].set_title(TAG + ", " + LABEL[u] + ", RSSI<=22")
            axs[u].set_ylabel("Temperature (C)")
            axs[u].set_xlabel("Time (s)")
            axs[u].xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
            axs[u].legend()

fig.subplots_adjust(hspace=0.4)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 13))
axs = [ax1, ax2, ax3]

TAGS = ["2B42", "3943", "3242"]

for u in range(3):
    for k,v in temp23p3.items():
        if k[-4:]==TAGS[u]:
            print(k)
            st, sr = sorted( temp23p3[k].items() ), sorted( rssi23p3[k].items() )
            dt, tt, dr = [ j[1] for j in st ], [ j[0] for j in st ], [ j[1] for j in sr ]

            newt = []
            newr = []
            newtime = []
            for r_,t_,x__ in zip(dr, dt, tt):
                if r_<=22:
                    newt.append(t_)
                    newr.append(r_)
                    newtime.append(x__)

            newtime, newr, newt = zip( *sorted( zip(newtime, newr, newt) ) )

            axs[u].plot(newtime, newt, color='#0d8aff', label='Raw Measured Temp', linewidth=0.65)

            measured_x, measured_y = TimeMoveAvg(newtime, newt, 10)
            actual_x, actual_y     = TimeMoveAvg(newtime, Actual(newt, newr, TAG), 10)
            
            # axs[u].plot(measured_x, measured_y, color='#021BE8', label='Measured Temp', linewidth=0.9)
            axs[u].plot(actual_x, actual_y,     color='#EB1213', label='Actual Temp, 10s Avg, 9s overlap with RSSI<21',   linewidth=1.05)

            axs[u].set_title(TAGS[u] + ", 23.3 #1" + ", RSSI<=22")
            axs[u].set_ylabel("Temperature (C)")
            axs[u].set_xlabel("Time (s)")
            axs[u].xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
            axs[u].legend()

fig.subplots_adjust(hspace=0.3)